In [1]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

100% 228M/228M [00:04<00:00, 92.7MB/s]



In [0]:
import zipfile
zip_ref = zipfile.ZipFile('the-movies-dataset.zip','r')
zip_ref.extractall('movie_data_set')
zip_ref.close()

In [0]:
import pandas as pd
import numpy as np

In [7]:
movie_data = pd.read_csv('movie_data_set/movies_metadata.csv')
credit_data = pd.read_csv('movie_data_set/credits.csv')
keyword_data = pd.read_csv('movie_data_set/keywords.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
def to_int(id):
    try:
        return np.int(id)
    except:
        return np.nan
   

In [0]:
movie_data['id'] = movie_data['id'].apply(to_int)
keyword_data['id'] = keyword_data['id'].apply(to_int)
credit_data['id'] = credit_data['id'].apply(to_int)

In [0]:
movie_data = movie_data.merge(credit_data, on='id')
movie_data =movie_data.merge(keyword_data, on='id')

In [0]:
del keyword_data
del credit_data

In [0]:
from ast import literal_eval
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    movie_data[feature] = movie_data[feature].apply(literal_eval)

In [13]:
movie_data.iloc[0]['crew'][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [0]:
def get_director(x):
    for crew_member in x:
        if crew_member['job'] == 'Director':
            return crew_member['name']
        return np.nan

In [0]:
movie_data['director'] = movie_data['crew'].apply(get_director)

In [0]:
def generate_list(x):
    if isinstance(x, list):
        names = [ele['name'] for ele in x]
        if len(names) > 3:
            names = names[:3]
        return names
    return []

In [0]:
movie_data['cast'] = movie_data['cast'].apply(generate_list)
movie_data['keywords'] = movie_data['keywords'].apply(generate_list)

In [0]:
movie_data['genres'] = movie_data['genres'].apply(generate_list)


In [0]:
movie_data = movie_data[['id','title','genres','keywords','cast','director']]


In [0]:
movie_data = movie_data.iloc[:25000,:]

In [0]:
#remove spaces
def sanitize(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    elif isinstance(x, str):
        return str.lower(x.replace(" ", ""))
    else:
        return ''

In [0]:
for feature in ['cast', 'director', 'genres', 'keywords']:
    movie_data[feature] = movie_data[feature].apply(sanitize)

In [0]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [0]:
movie_data['soup'] = movie_data.iloc[:,2:].apply(create_soup, axis=1)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(movie_data['soup'])

In [26]:
count_matrix.shape

(25000, 41190)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [0]:
indices = pd.Series(movie_data.index, index=movie_data['title'])

In [0]:
def content_recommender(title, cosine_sim=cosine_sim, data=movie_data, indices=indices):

    idx = indices[title]


    sim_scores = list(enumerate(cosine_sim[idx]))
#     print(sim_scores)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     print(sim_scores)
    sim_scores = sim_scores[1:11]
#     print(sim_scores)
    movie_indices = [i[0] for i in sim_scores]
#     print(movie_indices)
    return data['title'].iloc[movie_indices]

In [30]:
content_recommender('Jurassic Park')

1494            The Lost World: Jurassic Park
4550                        Jurassic Park III
767                          Independence Day
3833           Creature from the Black Lagoon
8426                     Island of Lost Souls
23320                           Night Gallery
1086               E.T. the Extra-Terrestrial
5747                        Mysterious Island
18223                      Radioactive Dreams
8889     The Hitch Hikers Guide to the Galaxy
Name: title, dtype: object